In [3]:
from lib import db_config
import plotly_express as px

import pandas as pd
import string
import re

# Mapping cohorts without looking at the PANs, but year and papercodes

In [4]:
# reading Dipak's data dump
dtype = {
 'school_code' : int,
 'school_name' : str,
 'city' : str,
 'pincode' : str,
 'state' : str,
 'country' : str,
 'season' : str,
 'year' : str,
 'class': np.uint8,
 'studentCount': int,
 'subject' : str,
 'papercode': str,
 'skill_name' : str,
 'sub_skill_name' : str,
 'subSkillLevelAccuracy': float,
 'subSkillLevelQuestionCount' : np.uint32,
 'skillLevelAccuracy' : float,
 'skillLevelQuestionCount' : np.uint32,
 'totalAccuracy' : float,
 'totalQuestionCount': np.uint32,
 'national_studentCount': int,
 'national_subSkillLevelAccuracy' : float,
 'totalStandardDeviation' : float,
 'national_skillLevelAccuracy' : float,
 'national_studentCount' : int,
 'national_totalAccuracy' : float,
 'national_totalStandardDeviation' : float,
 'national_skillLevelStandardDeviation': float}
file_path = '../../../Desktop/EI/From others/Dipak/Parent Connect/data2'
df = pd.read_csv(file_path+'/final_asset_school_accuracy.csv', dtype = dtype)

df['total_delta'] = df['totalAccuracy'] - df['national_totalAccuracy']
df['skill_delta'] = df['skillLevelAccuracy'] - df['national_skillLevelAccuracy']
df['subSkill_delta'] = df['subSkillLevelAccuracy'] - df['national_subSkillLevelAccuracy']

#Mapping to boards
df_school_board = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR_AO8CCe9wrCqYyXAKutE4NceqSuTZuWQdaZoxhLwPOBQYAXOIEZzXL1xsYMWX3uzhi9rtbNWk67PW/pub?gid=288331316&single=true&output=csv')
df = pd.merge(df, df_school_board, on='school_code', how = 'left')

In [40]:
#Choosing the school to analyse

# school_code = 173428
school_code = 46067
min_cohort_length = 4
df_school = df[df['school_code'] == school_code]

school_name = df_school.school_name.value_counts().index[0]
school_board = df_school.primary_board.value_counts().index[0]
school_city = df_school.city.value_counts().index[0]

In [6]:
## setup for t
# rounds = db_config.find_rounds(df_school)
# subjects = db_config.find_subjects(df_school)
# classes = db_config.find_classes(df_school)
# class_rounds = db_config.find_class_rounds(df_school)

# class_rounds.sort(reverse=True)
# class_rounds

# # ordered list of rounds
# r = list(string.digits)
# r.remove('0') 
# r += list(string.ascii_uppercase) + ['#']

# #ordered list of classes
# c = list(string.digits) + ['A', "B"]
# c.remove('0')

This funciton finds the number of cohorts that have likelyhood of having the same students

In [7]:
## This has been converted into a function
# # ordered list of rounds
# r = list(string.digits)
# r.remove('0') 
# r += list(string.ascii_uppercase) + ['#']

# #ordered list of classes
# c = list(string.digits) + ['A', "B"]
# c.remove('0')

# class_rounds = db_config.find_class_rounds(df_school)
# class_rounds.sort(reverse=True)

# temp = class_rounds
# cohorts = []
# cohort_long = []
# for i in temp:
#     cls, rnd = i
#     temp_cls = c[c.index(cls):]
#     temp_rnd = r[r.index(rnd)::2]
#     co = list(zip(temp_cls, temp_rnd))
#     co = [''.join(x) for x in co]

#     cohort = [x for x in co if x in class_rounds]
# #     print( co, cohort)
#     cohorts.append(cohort)
# #     print(cohorts,"--" )
#     temp = list(set(temp) - set(cohort))
# #     break

# cohort_long = [x for x in cohorts if len(x) > 1]

# cohort_long

In [41]:
# takes a dataframe of a a school's data and finds trains of cohorts

def find_cohorts(df_school):
    # ordered list of rounds
    r = list(string.digits)
    r.remove('0') 
    r += list(string.ascii_uppercase) + ['#']

    #ordered list of classes
    c = list(string.digits) + ['A', "B"]
    c.remove('0')
    
    #combinations of tests in class rounds in that school that has happened
    class_rounds = db_config.find_class_rounds(df_school)
#     class_rounds.sort(reverse=True)
    class_rounds.sort(reverse=False)
    temp = class_rounds
    cohorts = []
    cohort_long = []
    for i in temp:
        cls, rnd = i
        temp_cls = c[c.index(cls):]
        temp_rnd = r[r.index(rnd)::2]
        co = list(zip(temp_cls, temp_rnd))
        co = [''.join(x) for x in co]

        cohort = [x for x in co if x in class_rounds]
    #     print( co, cohort)
        cohorts.append(cohort)
    #     print(cohorts,"--" )
        temp = list(set(temp) - set(cohort))
#     removing the duplicates
    l = cohorts
    l2 = l[:]
    for m in l:
        for n in l:
            if set(m).issubset(set(n)) and m != n:
                l2.remove(m)
                break

    return [x for x in l2 if len(x) > min_cohort_length]

In [42]:
cohorts = find_cohorts(df_school)

In [43]:
cohorts

[['37', '49', '5B', '6D', '7F', '8H', '9J'],
 ['39', '4B', '5D', '6F', '7H', '8J', 'AN'],
 ['3B', '4D', '5F', '6H', '7J', '9N', 'AP'],
 ['3D', '4F', '5H', '6J', '7L', '8N', '9P', 'AR'],
 ['3F', '4H', '5J', '7N', '8P', '9R', 'AT'],
 ['3H', '4J', '6N', '7P', '8R', '9T', 'AV'],
 ['3J', '5N', '6P', '7R', '8T', '9V'],
 ['3N', '4P', '5R', '6T', '7V', '9Z'],
 ['3P', '4R', '5T', '6V', '8Z'],
 ['47', '59', '6B', '7D', '8F', '9H'],
 ['4N', '5P', '6R', '7T', '8V'],
 ['57', '69', '7B', '8D', '9F']]

In [36]:
# # one way of selecting the table
# df_school.papercode[df_school.papercode.str.contains('|'.join(cohorts[9]))].value_counts()

In [38]:
## testing the function to be able to genreate plot of the progress

# i = ['8S', 'AW']
i = cohorts[0]
# forms a regex to choose the appropriate papercodes
j = lambda x :'.'+'('+'|'.join(x)+')'
# returns true or false based on whether the paper code contains the cohort
present = lambda x : bool(re.search(j(i), x))

temp = df_school[df_school.papercode.apply(present)].drop_duplicates(['papercode', 'school_code'])
temp.year = temp.year.astype(str)
px.scatter(
    temp,
    x = 'year',
    y = 'total_delta',
    color = 'subject',
    size = 'studentCount',
    text = ['class', 'subject'], 
    trendline = 'lowess',
#     range_x = [2007, 2019],
    title = "Year on Year performance of same students as they progress across the years"
)
#     break

Looping over the data to label the cohorts in each school

In [44]:
data, temp = pd.DataFrame(),pd.DataFrame()
for ix, i in enumerate(cohorts):
    # forms a regex to choose the appropriate papercodes
    j = lambda x :'.'+'('+'|'.join(x)+')'
    # returns true or false based on whether the paper code contains the cohort
    present = lambda x : bool(re.search(j(i), x))
    temp = df_school[df_school.papercode.apply(present)]
    temp.year = temp.year.astype(str)
    temp['cohort_number'] = ix +1
    temp.cohort_number = temp.cohort_number.astype(int)
    data  = data.append(temp)

/Users/manghat/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/manghat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [46]:
for i, j in enumerate(cohorts):
    print("Cohort Number :", i+1, "Rounds :", j)
px.scatter(
    data.sort_values('year'),#.drop_duplicates(['papercode']),
    x = 'year', 
#     range_x = [2001,2019],
#     range_y = [-10, 30],
    y = 'total_delta',
    color = 'subject',
    facet_row = 'cohort_number',
    size = 'studentCount',
    text = 'class', 
    trendline = 'lowless',
    title = "Year on Year performance of same students in %s as they progress across the years"%school_name,
    height = 5000,
    size_max = 10
)

Cohort Number : 1 Rounds : ['37', '49', '5B', '6D', '7F', '8H', '9J']
Cohort Number : 2 Rounds : ['39', '4B', '5D', '6F', '7H', '8J', 'AN']
Cohort Number : 3 Rounds : ['3B', '4D', '5F', '6H', '7J', '9N', 'AP']
Cohort Number : 4 Rounds : ['3D', '4F', '5H', '6J', '7L', '8N', '9P', 'AR']
Cohort Number : 5 Rounds : ['3F', '4H', '5J', '7N', '8P', '9R', 'AT']
Cohort Number : 6 Rounds : ['3H', '4J', '6N', '7P', '8R', '9T', 'AV']
Cohort Number : 7 Rounds : ['3J', '5N', '6P', '7R', '8T', '9V']
Cohort Number : 8 Rounds : ['3N', '4P', '5R', '6T', '7V', '9Z']
Cohort Number : 9 Rounds : ['3P', '4R', '5T', '6V', '8Z']
Cohort Number : 10 Rounds : ['47', '59', '6B', '7D', '8F', '9H']
Cohort Number : 11 Rounds : ['4N', '5P', '6R', '7T', '8V']
Cohort Number : 12 Rounds : ['57', '69', '7B', '8D', '9F']


# TODO 
- loop around the cohorts to plot it all in the same graph - done


In [28]:
c = [set(x) for x in cohorts ]

In [31]:
l = cohorts
l2 = l[:]
for m in l:
    for n in l:
        if set(m).issubset(set(n)) and m != n:
            l2.remove(m)
            break

In [32]:
l2

[['37', '49', '5B', '6D', '7F', '8H', '9J'],
 ['38', '6E', '7G'],
 ['39', '4B', '5D', '6F', '7H', '8J', 'AN'],
 ['3B', '4D', '5F', '6H', '7J', '9N', 'AP'],
 ['3D', '4F', '5H', '6J', '7L', '8N', '9P', 'AR'],
 ['3E', '4G', '7M'],
 ['3F', '4H', '5J', '7N', '8P', '9R', 'AT'],
 ['3G', '6M'],
 ['3H', '4J', '6N', '7P', '8R', '9T', 'AV'],
 ['3J', '5N', '6P', '7R', '8T', '9V'],
 ['3M', '9Y'],
 ['3N', '4P', '5R', '6T', '7V', '9Z'],
 ['3P', '4R', '5T', '6V', '8Z'],
 ['3R', '4T', '5V', '7Z'],
 ['3T', '4V', '6Z'],
 ['3V', '5Z'],
 ['47', '59', '6B', '7D', '8F', '9H'],
 ['48', '7E', '8G'],
 ['4E', '5G', '8M'],
 ['4N', '5P', '6R', '7T', '8V'],
 ['57', '69', '7B', '8D', '9F'],
 ['5E', '6G', '9M'],
 ['67', '79', '8B', '9D'],
 ['77', '89', '9B'],
 ['8E', '9G']]